In [154]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [57]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras import optimizers
from keras.utils import plot_model
from keras.models import Sequential, Model
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Flatten
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

%matplotlib inline
warnings.filterwarnings("ignore")
init_notebook_mode(connected=True)

# Set seeds to make the experiment more reproducible.
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(1)

In [58]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from plotly.offline import init_notebook_mode, iplot
import chart_studio.plotly as py
import plotly.graph_objs as go

%matplotlib inline
warnings.filterwarnings("ignore")
init_notebook_mode(connected=True)

In [59]:
train = pd.read_csv('train.csv', parse_dates=['date'],index_col=['date'])


In [60]:
train

,store,item,sales
date,,,
2013-01-01,1,1,13
2013-01-02,1,1,11
2013-01-03,1,1,14
2013-01-04,1,1,13
2013-01-05,1,1,10
...,...,...,...
2017-12-27,10,50,63
2017-12-28,10,50,59
2017-12-29,10,50,74


In [61]:
train.dtypes

store    int64
item     int64
sales    int64
dtype: object

In [113]:
test = pd.read_csv('test.csv', parse_dates=['date'],index_col=['date'])

In [114]:
train['day']=train.index.day
train['month']=train.index.month
train['year']=train.index.year
train['dayofweek']=train.index.dayofweek
train['weekofyear']=train.index.isocalendar().week
train['is_weekend']=train.index.dayofweek

In [115]:
test['day'] = test.index.day
test['month'] = test.index.month
test['year'] = test.index.year
test['dayofweek'] = test.index.dayofweek
test['weekofyear']  = test.index.isocalendar().week
test['is_weekend'] = test.index.dayofweek 

In [65]:
train

,store,item,sales,day,month,year,dayofweek,weekofyear,is_weekend
date,,,,,,,,,
2013-01-01,1,1,13,1,1,2013,1,1,1
2013-01-02,1,1,11,2,1,2013,2,1,2
2013-01-03,1,1,14,3,1,2013,3,1,3
2013-01-04,1,1,13,4,1,2013,4,1,4
2013-01-05,1,1,10,5,1,2013,5,1,5
...,...,...,...,...,...,...,...,...,...
2017-12-27,10,50,63,27,12,2017,2,52,2
2017-12-28,10,50,59,28,12,2017,3,52,3
2017-12-29,10,50,74,29,12,2017,4,52,4


In [66]:
train_not_17 = train[train['year'] != 2017]

In [67]:
train_not_17

,store,item,sales,day,month,year,dayofweek,weekofyear,is_weekend
date,,,,,,,,,
2013-01-01,1,1,13,1,1,2013,1,1,1
2013-01-02,1,1,11,2,1,2013,2,1,2
2013-01-03,1,1,14,3,1,2013,3,1,3
2013-01-04,1,1,13,4,1,2013,4,1,4
2013-01-05,1,1,10,5,1,2013,5,1,5
...,...,...,...,...,...,...,...,...,...
2016-12-27,10,50,60,27,12,2016,1,52,1
2016-12-28,10,50,43,28,12,2016,2,52,2
2016-12-29,10,50,68,29,12,2016,3,52,3


In [68]:
Y = train_not_17['sales']
Y

date
2013-01-01    13
2013-01-02    11
2013-01-03    14
2013-01-04    13
2013-01-05    10
              ..
2016-12-27    60
2016-12-28    43
2016-12-29    68
2016-12-30    63
2016-12-31    64
Name: sales, Length: 730500, dtype: int64

In [69]:
X = train_not_17.drop(columns='sales')

In [70]:
X

,store,item,day,month,year,dayofweek,weekofyear,is_weekend
date,,,,,,,,
2013-01-01,1,1,1,1,2013,1,1,1
2013-01-02,1,1,2,1,2013,2,1,2
2013-01-03,1,1,3,1,2013,3,1,3
2013-01-04,1,1,4,1,2013,4,1,4
2013-01-05,1,1,5,1,2013,5,1,5
...,...,...,...,...,...,...,...,...
2016-12-27,10,50,27,12,2016,1,52,1
2016-12-28,10,50,28,12,2016,2,52,2
2016-12-29,10,50,29,12,2016,3,52,3


In [71]:
X.reset_index(drop=True, inplace=True)

In [72]:
X

,store,item,day,month,year,dayofweek,weekofyear,is_weekend
0,1,1,1,1,2013,1,1,1
1,1,1,2,1,2013,2,1,2
2,1,1,3,1,2013,3,1,3
3,1,1,4,1,2013,4,1,4
4,1,1,5,1,2013,5,1,5
...,...,...,...,...,...,...,...,...
730495,10,50,27,12,2016,1,52,1
730496,10,50,28,12,2016,2,52,2
730497,10,50,29,12,2016,3,52,3
730498,10,50,30,12,2016,4,52,4


In [73]:
X.shape[1]

8

In [74]:
epochs = 40
batch = 256
lr = 0.0003
adam = optimizers.Adam(lr)

In [75]:
from keras.layers import LSTM,SimpleRNN

In [76]:
n_cols= X.shape[1]

In [77]:
model_mlp = Sequential()
model_mlp.add(Dense(100, activation='relu', input_dim=X.shape[1]))
model_mlp.add(Dense(1))
model_mlp.compile(loss='mse', optimizer=adam)
model_mlp.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 100)               900       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 1,001
Trainable params: 1,001
Non-trainable params: 0
_________________________________________________________________


In [78]:
import numpy as np

X = np.asarray(X).astype(np.float32)

In [79]:

#set early stopping monitor so the model stops training when it won't improve anymore

#train model
model_mlp.fit(X, Y , epochs=epochs, verbose=2)

Epoch 1/40
22829/22829 - 31s - loss: 790.7649
Epoch 2/40
22829/22829 - 28s - loss: 611.2477
Epoch 3/40
22829/22829 - 27s - loss: 604.4960
Epoch 4/40
22829/22829 - 27s - loss: 600.1037
Epoch 5/40
22829/22829 - 28s - loss: 597.2694
Epoch 6/40
22829/22829 - 27s - loss: 594.9983
Epoch 7/40
22829/22829 - 27s - loss: 592.7421
Epoch 8/40
22829/22829 - 28s - loss: 591.8452
Epoch 9/40
22829/22829 - 32s - loss: 590.4196
Epoch 10/40
22829/22829 - 32s - loss: 589.2462
Epoch 11/40
22829/22829 - 33s - loss: 587.7325
Epoch 12/40
22829/22829 - 31s - loss: 587.2256
Epoch 13/40
22829/22829 - 29s - loss: 586.1198
Epoch 14/40
22829/22829 - 31s - loss: 585.5578
Epoch 15/40
22829/22829 - 30s - loss: 584.7051
Epoch 16/40
22829/22829 - 31s - loss: 584.2814
Epoch 17/40
22829/22829 - 30s - loss: 583.9628
Epoch 18/40
22829/22829 - 29s - loss: 583.5078
Epoch 19/40
22829/22829 - 30s - loss: 583.0607
Epoch 20/40
22829/22829 - 30s - loss: 582.9534
Epoch 21/40
22829/22829 - 30s - loss: 582.5441
Epoch 22/40
22829/2282

In [80]:
train

,store,item,sales,day,month,year,dayofweek,weekofyear,is_weekend
date,,,,,,,,,
2013-01-01,1,1,13,1,1,2013,1,1,1
2013-01-02,1,1,11,2,1,2013,2,1,2
2013-01-03,1,1,14,3,1,2013,3,1,3
2013-01-04,1,1,13,4,1,2013,4,1,4
2013-01-05,1,1,10,5,1,2013,5,1,5
...,...,...,...,...,...,...,...,...,...
2017-12-27,10,50,63,27,12,2017,2,52,2
2017-12-28,10,50,59,28,12,2017,3,52,3
2017-12-29,10,50,74,29,12,2017,4,52,4


In [81]:
train_17 = train[train['year'] == 2017]

In [82]:
train_17

,store,item,sales,day,month,year,dayofweek,weekofyear,is_weekend
date,,,,,,,,,
2017-01-01,1,1,19,1,1,2017,6,52,6
2017-01-02,1,1,15,2,1,2017,0,1,0
2017-01-03,1,1,10,3,1,2017,1,1,1
2017-01-04,1,1,16,4,1,2017,2,1,2
2017-01-05,1,1,14,5,1,2017,3,1,3
...,...,...,...,...,...,...,...,...,...
2017-12-27,10,50,63,27,12,2017,2,52,2
2017-12-28,10,50,59,28,12,2017,3,52,3
2017-12-29,10,50,74,29,12,2017,4,52,4


In [83]:
X1 = train_17.drop(columns='sales')
X1

,store,item,day,month,year,dayofweek,weekofyear,is_weekend
date,,,,,,,,
2017-01-01,1,1,1,1,2017,6,52,6
2017-01-02,1,1,2,1,2017,0,1,0
2017-01-03,1,1,3,1,2017,1,1,1
2017-01-04,1,1,4,1,2017,2,1,2
2017-01-05,1,1,5,1,2017,3,1,3
...,...,...,...,...,...,...,...,...
2017-12-27,10,50,27,12,2017,2,52,2
2017-12-28,10,50,28,12,2017,3,52,3
2017-12-29,10,50,29,12,2017,4,52,4


In [84]:
X1.reset_index(drop=True, inplace=True)
X1

,store,item,day,month,year,dayofweek,weekofyear,is_weekend
0,1,1,1,1,2017,6,52,6
1,1,1,2,1,2017,0,1,0
2,1,1,3,1,2017,1,1,1
3,1,1,4,1,2017,2,1,2
4,1,1,5,1,2017,3,1,3
...,...,...,...,...,...,...,...,...
182495,10,50,27,12,2017,2,52,2
182496,10,50,28,12,2017,3,52,3
182497,10,50,29,12,2017,4,52,4
182498,10,50,30,12,2017,5,52,5


In [85]:
Y1 = train_17['sales']

In [86]:
Y1

date
2017-01-01    19
2017-01-02    15
2017-01-03    10
2017-01-04    16
2017-01-05    14
              ..
2017-12-27    63
2017-12-28    59
2017-12-29    74
2017-12-30    62
2017-12-31    82
Name: sales, Length: 182500, dtype: int64

In [87]:
Train_Pred = model_mlp.predict(X)

In [88]:
Train_Pred

array([[11.937935],
       [14.836906],
       [17.689346],
       ...,
       [38.47689 ],
       [41.760124],
       [45.043358]], dtype=float32)

In [89]:
print('Train rmse:', np.sqrt(mean_squared_error(Y,Train_Pred)))
print('Train mape:', mean_absolute_error(Y,Train_Pred))

Train rmse: 23.685396991695644
Train mape: 19.24639427397228


In [90]:
X1

,store,item,day,month,year,dayofweek,weekofyear,is_weekend
0,1,1,1,1,2017,6,52,6
1,1,1,2,1,2017,0,1,0
2,1,1,3,1,2017,1,1,1
3,1,1,4,1,2017,2,1,2
4,1,1,5,1,2017,3,1,3
...,...,...,...,...,...,...,...,...
182495,10,50,27,12,2017,2,52,2
182496,10,50,28,12,2017,3,52,3
182497,10,50,29,12,2017,4,52,4
182498,10,50,30,12,2017,5,52,5


In [94]:
X1 = np.asarray(X1).astype(np.float32)

In [95]:
Pred = model_mlp.predict(X1)

In [96]:
Pred

array([[29.439423],
       [ 8.768234],
       [11.667213],
       ...,
       [41.657005],
       [44.940235],
       [48.223465]], dtype=float32)

In [97]:
len(Pred)

182500

In [98]:
print('Validation rmse:', np.sqrt(mean_squared_error(Y1,Pred)))

Train rmse: 27.059341433337952


In [99]:
from sklearn.metrics import mean_absolute_error

In [100]:
print('Validation mape:', mean_absolute_error(Y1,Pred))

Train mape: 21.230704030687352


In [147]:
def mean_absolute_percentage_error(y_true, y_pred): 

    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [150]:
pred = Pred.ravel().tolist()

In [152]:
mean_absolute_percentage_error(Y1,pred)

45.50415600572672

In [139]:
MAPE_1_MLP = mean_absolute_error(Y1,Pred)

In [140]:
MAPE_1_MLP

21.230704030687352

In [116]:
test

,id,store,item,day,month,year,dayofweek,weekofyear,is_weekend
date,,,,,,,,,
2018-01-01,0,1,1,1,1,2018,0,1,0
2018-01-02,1,1,1,2,1,2018,1,1,1
2018-01-03,2,1,1,3,1,2018,2,1,2
2018-01-04,3,1,1,4,1,2018,3,1,3
2018-01-05,4,1,1,5,1,2018,4,1,4
...,...,...,...,...,...,...,...,...,...
2018-03-27,44995,10,50,27,3,2018,1,13,1
2018-03-28,44996,10,50,28,3,2018,2,13,2
2018-03-29,44997,10,50,29,3,2018,3,13,3


In [117]:
test.reset_index(drop=True, inplace=True)

In [118]:
test

,id,store,item,day,month,year,dayofweek,weekofyear,is_weekend
0,0,1,1,1,1,2018,0,1,0
1,1,1,1,2,1,2018,1,1,1
2,2,1,1,3,1,2018,2,1,2
3,3,1,1,4,1,2018,3,1,3
4,4,1,1,5,1,2018,4,1,4
...,...,...,...,...,...,...,...,...,...
44995,44995,10,50,27,3,2018,1,13,1
44996,44996,10,50,28,3,2018,2,13,2
44997,44997,10,50,29,3,2018,3,13,3
44998,44998,10,50,30,3,2018,4,13,4


In [119]:
test.drop(columns='id',inplace=True)

In [120]:
test

,store,item,day,month,year,dayofweek,weekofyear,is_weekend
0,1,1,1,1,2018,0,1,0
1,1,1,2,1,2018,1,1,1
2,1,1,3,1,2018,2,1,2
3,1,1,4,1,2018,3,1,3
4,1,1,5,1,2018,4,1,4
...,...,...,...,...,...,...,...,...
44995,10,50,27,3,2018,1,13,1
44996,10,50,28,3,2018,2,13,2
44997,10,50,29,3,2018,3,13,3
44998,10,50,30,3,2018,4,13,4


In [121]:
test = np.asarray(test).astype(np.float32)

In [122]:
Test_Pred = model_mlp.predict(test)

In [123]:
Test_Pred

array([[ 8.9109955],
       [11.809944 ],
       [14.708916 ],
       ...,
       [45.87796  ],
       [48.014225 ],
       [50.150497 ]], dtype=float32)

In [ ]:
c = Test_Pred.ravel().tolist()

In [146]:
len(c)

45000

ValueError: operands could not be broadcast together with shapes (182500,) (45000,) 

In [133]:
l1=[]
for i in c:
    l1.append(int(i))

In [134]:
l1

[8,
 11,
 14,
 17,
 19,
 22,
 24,
 9,
 11,
 14,
 17,
 20,
 22,
 24,
 9,
 12,
 15,
 17,
 20,
 22,
 24,
 9,
 12,
 15,
 17,
 20,
 22,
 24,
 9,
 12,
 15,
 24,
 26,
 29,
 31,
 15,
 18,
 21,
 24,
 27,
 29,
 31,
 15,
 18,
 21,
 24,
 27,
 29,
 31,
 15,
 18,
 21,
 24,
 27,
 29,
 32,
 15,
 18,
 21,
 30,
 32,
 35,
 38,
 21,
 24,
 27,
 30,
 33,
 36,
 38,
 21,
 24,
 27,
 30,
 33,
 36,
 38,
 21,
 24,
 27,
 30,
 33,
 36,
 38,
 21,
 24,
 27,
 30,
 33,
 36,
 7,
 9,
 12,
 15,
 17,
 20,
 22,
 7,
 10,
 13,
 15,
 18,
 20,
 22,
 7,
 10,
 13,
 15,
 18,
 20,
 22,
 7,
 10,
 13,
 15,
 18,
 20,
 22,
 7,
 10,
 13,
 22,
 25,
 27,
 29,
 13,
 16,
 19,
 22,
 25,
 27,
 29,
 13,
 16,
 19,
 22,
 25,
 27,
 30,
 13,
 16,
 19,
 22,
 25,
 27,
 30,
 14,
 16,
 19,
 28,
 31,
 34,
 36,
 19,
 22,
 25,
 28,
 31,
 34,
 36,
 19,
 22,
 25,
 28,
 31,
 34,
 36,
 19,
 22,
 25,
 28,
 31,
 34,
 36,
 20,
 23,
 25,
 28,
 31,
 34,
 5,
 8,
 11,
 13,
 16,
 18,
 20,
 5,
 8,
 11,
 13,
 16,
 18,
 20,
 5,
 8,
 11,
 13,
 16,
 18,
 20,
 5,
 8,
 11,

In [127]:
len(Test_Pred)

45000

In [129]:
Test_Pred[65]

array([27.317253], dtype=float32)

In [124]:
%store -r l 

In [125]:
l

[12,
 13,
 13,
 13,
 17,
 17,
 20,
 12,
 13,
 13,
 13,
 17,
 17,
 20,
 12,
 13,
 14,
 14,
 17,
 18,
 20,
 12,
 15,
 15,
 15,
 17,
 18,
 20,
 12,
 15,
 15,
 15,
 18,
 18,
 21,
 12,
 15,
 16,
 16,
 17,
 18,
 20,
 12,
 16,
 16,
 16,
 18,
 18,
 20,
 12,
 16,
 16,
 16,
 18,
 18,
 20,
 12,
 16,
 16,
 17,
 18,
 17,
 19,
 14,
 15,
 17,
 17,
 18,
 18,
 19,
 15,
 17,
 19,
 19,
 20,
 20,
 21,
 15,
 17,
 18,
 18,
 20,
 20,
 21,
 16,
 16,
 17,
 17,
 19,
 19,
 17,
 19,
 19,
 20,
 23,
 23,
 23,
 17,
 19,
 19,
 20,
 23,
 23,
 23,
 17,
 19,
 19,
 20,
 23,
 22,
 23,
 17,
 20,
 20,
 21,
 24,
 23,
 23,
 16,
 20,
 20,
 24,
 26,
 27,
 28,
 17,
 23,
 23,
 24,
 27,
 28,
 29,
 17,
 22,
 23,
 23,
 27,
 28,
 29,
 17,
 20,
 20,
 21,
 27,
 28,
 29,
 17,
 20,
 20,
 26,
 31,
 33,
 33,
 21,
 24,
 25,
 26,
 31,
 32,
 32,
 21,
 24,
 24,
 25,
 28,
 29,
 29,
 21,
 25,
 24,
 25,
 29,
 30,
 30,
 21,
 25,
 24,
 25,
 30,
 30,
 16,
 19,
 19,
 20,
 19,
 21,
 20,
 16,
 19,
 19,
 20,
 19,
 21,
 20,
 16,
 19,
 19,
 20,
 19,
 20,


In [128]:
len(l)

45000

In [137]:
diff = [a - b for a, b in zip(l, l1)]

In [138]:
diff

[4,
 2,
 -1,
 -4,
 -2,
 -5,
 -4,
 3,
 2,
 -1,
 -4,
 -3,
 -5,
 -4,
 3,
 1,
 -1,
 -3,
 -3,
 -4,
 -4,
 3,
 3,
 0,
 -2,
 -3,
 -4,
 -4,
 3,
 3,
 0,
 -9,
 -8,
 -11,
 -10,
 -3,
 -3,
 -5,
 -8,
 -10,
 -11,
 -11,
 -3,
 -2,
 -5,
 -8,
 -9,
 -11,
 -11,
 -3,
 -2,
 -5,
 -8,
 -9,
 -11,
 -12,
 -3,
 -2,
 -5,
 -13,
 -14,
 -18,
 -19,
 -7,
 -9,
 -10,
 -13,
 -15,
 -18,
 -19,
 -6,
 -7,
 -8,
 -11,
 -13,
 -16,
 -17,
 -6,
 -7,
 -9,
 -12,
 -13,
 -16,
 -17,
 -5,
 -8,
 -10,
 -13,
 -14,
 -17,
 10,
 10,
 7,
 5,
 6,
 3,
 1,
 10,
 9,
 6,
 5,
 5,
 3,
 1,
 10,
 9,
 6,
 5,
 5,
 2,
 1,
 10,
 10,
 7,
 6,
 6,
 3,
 1,
 9,
 10,
 7,
 2,
 1,
 0,
 -1,
 4,
 7,
 4,
 2,
 2,
 1,
 0,
 4,
 6,
 4,
 1,
 2,
 1,
 -1,
 4,
 4,
 1,
 -1,
 2,
 1,
 -1,
 3,
 4,
 1,
 -2,
 0,
 -1,
 -3,
 2,
 2,
 0,
 -2,
 0,
 -2,
 -4,
 2,
 2,
 -1,
 -3,
 -3,
 -5,
 -7,
 2,
 3,
 -1,
 -3,
 -2,
 -4,
 -6,
 1,
 2,
 -1,
 -3,
 -1,
 -4,
 11,
 11,
 8,
 7,
 3,
 3,
 0,
 11,
 11,
 8,
 7,
 3,
 3,
 0,
 11,
 11,
 8,
 7,
 3,
 2,
 0,
 11,
 11,
 9,
 8,
 3,
 2,
 -1,
 11,
 11,
 9,
 3,
 1

In [143]:
import pickle
## open a file , where you want to store the data
file = open('model_1_mlp.pkl','wb')

##dump information to that file
pickle.dump(model_mlp, file)

TypeError: can't pickle _thread.RLock objects

In [ ]:
https://towardsdatascience.com/which-evaluation-metric-should-you-use-in-machine-learning-regression-problems-20cdaef258e

In [153]:
model_mlp.save("MLP_Model")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: MLP_Model\assets
